In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
def hbr_srch(queries:list, pages:int=1) ->pd.DataFrame:

  df = pd.DataFrame()
  articles_id = set()
  for query in queries:
    for p in range(1,pages+1):
      time.sleep(0.5)
      url = 'https://habr.com'
      params = {
          'q': query
      }
      headers = {
          'Accept-Language': 'ru,en-us;q=0.7,en;q=0.3'
      }
      with requests.get(url+f'/search/page{p}',params=params, headers=headers) as response:
        soup = BeautifulSoup(response.text)

      articles = soup.find_all('article', 'tm-articles-list__item') 
      rows = []

      for elem in articles:
        if elem.get('id') in articles_id:
          continue

        if is_megapost:= elem.find('div','tm-megapost-snippet'):
          print('megapost')
          articles_id.add(elem.get('id'))
          data = pd.to_datetime(elem.find('time').get('datetime'))
          title = elem.find('h2', 'tm-megapost-snippet__title').text
          link = url + elem.find('a', 'tm-megapost-snippet__link').get('href')
          likes = elem.find('span', 'tm-votes-meter__value').text
          print(link)

          with requests.get(link) as res:
            is_200 = res.status_code
            print(is_200)
            inner_soup = BeautifulSoup(res.text)
          
          text_content = inner_soup.find(id='post-content-body').text.strip().replace('\n','')
        
        else:

          articles_id.add(elem.get('id'))
          data = pd.to_datetime(elem.find('time').get('datetime'))
          title = elem.find('h2').find('span').text
          link = url + elem.find('h2').find('a').get('href')
          likes = elem.find('span', 'tm-votes-meter__value').text
          print(link)

          if link:  # обработка постов с удаленных ресурсов при условии что у них такой же тег article и класс tm-articles-list__item
                    # если же они отличаются, то они пропускаются, но мне кажется я на них не наткнулся
            with requests.get(link) as res:
              is_200 = res.status_code
              print(is_200)
              inner_soup = BeautifulSoup(res.text)

            not_found = inner_soup.find('div', 'tm-error-message__body')

            if is_200 == 404:
              text_content = not_found.text.strip().replace('\n','')

            elif is_expired:=inner_soup.find('p', 'tm-expired-company__text'):
              text_content = is_expired.text.strip().replace('\n','')

            elif is_200 == 200:
              text_content = inner_soup.find('div','article-formatted-body').text.strip().replace('\n','')

            else:
              text_content = 'Unexpected error'

          else: 
            print('remote')
            continue

        rows.append({'data':data, 'title': title, 'link': link, 'text_content':text_content, 'likes':likes})

      inner_df = pd.DataFrame(rows)
      df = pd.concat([df,inner_df], ignore_index=True)

  return df

In [6]:
df = hbr_srch([''],1)

In [8]:
pd.set_option('display.max_rows',100)
df.head(100)

""
